In [1]:
import numpy as np
import scipy.stats as sps
from scipy.stats import ttest_ind, mannwhitneyu

# Part 1

Реализовать t-критерий Стьюдента для 2 независимых выборок при условии неизвестной дисперсии. Необходимо считать значение статистики и p_value для разных видов гипотез (двусторонняя, односторонние), а также результат (отвергается нулевая гипотеза или нет). Сравнить результаты с реализацией в scipy.stats.ttest_ind.

In [2]:
def ttest_stat_counter(data1, data2, alpha, alternative='two-sided'):
    mean1 = np.mean(data1)
    mean2 = np.mean(data2)
    var1 = np.var(data1, ddof=1)
    var2 = np.var(data2, ddof=1)
    
    n1 = len(data1)
    n2 = len(data2)
    n = n1 + n2 - 2
    t_stat = (mean1 - mean2) / np.sqrt(var1 / n1 + var2 / n2)
    p_value = sps.t.cdf(t_stat, n)

    if alternative == 'two-sided':
        p_value *= 2

    elif alternative == 'greater':
        p_value = 1 - p_value
        
    #else 'less'    
    if p_value < alpha:
        result = 'Rejected'
    else:
        result = 'Approwed'
    
    return mean1, mean2, var1, var2, t_stat, p_value, result

In [3]:
Mu1 = 0
Sig1 = 1

Mu2 = 0.5
Sig2 = 1

np.random.seed(42)
data1 = np.random.normal(loc=Mu1, scale=Sig1, size=100)
data2 = np.random.normal(loc=Mu2, scale=Sig2, size=100)

alpha = 0.05

In [4]:
alternatives = ['two-sided', 'less', 'greater']
for alternative in alternatives:
    mean1, mean2, var1, var2, t_stat, p_value, hyp_res = ttest_stat_counter(data1, data2, alpha, alternative=alternative)
    t_stat_skipy, pvalue_skipy = ttest_ind(data1, data2, alternative=alternative)

    print(f"Hypothesis name: {alternative}")
    print("Mean #1:", round(mean1, 5))
    print("Mean #2:", round(mean2, 5))
    print("Variance 1:", round(var1, 5))
    print("Variance 2:", round(var2, 5))
    print("My t-statistic and scipy.stats:", round(t_stat, 5), round(t_stat_skipy, 5))
    print("My p-value implemented and scipy.stats:", round(p_value, 10), round(pvalue_skipy, 10))
    print("Hypothesis result:", hyp_res)
    print()

Hypothesis name: two-sided
Mean #1: -0.10385
Mean #2: 0.5223
Variance 1: 0.82477
Variance 2: 0.90948
My t-statistic and scipy.stats: -4.7547 -4.7547
My p-value implemented and scipy.stats: 3.8191e-06 3.8191e-06
Hypothesis result: Rejected

Hypothesis name: less
Mean #1: -0.10385
Mean #2: 0.5223
Variance 1: 0.82477
Variance 2: 0.90948
My t-statistic and scipy.stats: -4.7547 -4.7547
My p-value implemented and scipy.stats: 1.9096e-06 1.9096e-06
Hypothesis result: Rejected

Hypothesis name: greater
Mean #1: -0.10385
Mean #2: 0.5223
Variance 1: 0.82477
Variance 2: 0.90948
My t-statistic and scipy.stats: -4.7547 -4.7547
My p-value implemented and scipy.stats: 0.9999980904 0.9999980904
Hypothesis result: Approwed



# Part 2

Implement bootstrap for estimating using confidence intervals median and mean values of distribution.  Calculate for distributions: normal, exponential, normal mixture

In [5]:
import numpy as np

def bootstrap(data, statistic, n_bootstrap, alpha):
    n = len(data)
    boot_samples = np.random.choice(data, size=(n_bootstrap, n), replace=True)
    boot_statistics = np.zeros(n_bootstrap)
    
    for i in range(n_bootstrap):
        boot_statistics[i] = statistic(boot_samples[i])
    
    boot_statistics.sort()
    lower_idx = int(n_bootstrap * alpha / 2)
    upper_idx = int(n_bootstrap * (1 - alpha / 2))
    lower_bound = boot_statistics[lower_idx]
    upper_bound = boot_statistics[upper_idx]
    
    return lower_bound, upper_bound


In [6]:
# Normal Distribution
N_BOOTSTRAP = 1000
ALPHA = 0.05
SIZE = 1000
SCALE = 1
MU = 0
DELTA = 2
np.random.seed(42)

normal_data = np.random.normal(loc=MU, scale=SCALE, size=SIZE)
exp_data = np.random.exponential(scale=SCALE, size=SIZE)
mixed_normal_data = np.concatenate([np.random.normal(loc=MU - DELTA, scale=SCALE, size=SIZE // 2),
                                      np.random.normal(loc=MU + DELTA, scale=SCALE, size=SIZE // 2)])


distributions = [{'data': normal_data, 'name': 'normal'},
                 {'data': exp_data, 'name': 'exponential'},
                 {'data': mixed_normal_data, 'name': 'mixed normal'},
                ]
statistics = [{'func': np.mean, 'name': 'mean'},
              {'func': np.median, 'name': 'median'}
             ]

for distribution in distributions:
    for statistic in statistics:
        stat_lower, stat_upper = bootstrap(distribution['data'], statistic['func'], N_BOOTSTRAP, ALPHA)
        print(f'Distribution: {distribution["name"]}, statistic: {statistic["name"]}')
        print(f'Lower bound: {np.round(stat_lower, 5)}')
        print(f'Upper bound: {np.round(stat_upper, 5)}')
        print()



Distribution: normal, statistic: mean
Lower bound: -0.04092
Upper bound: 0.0822

Distribution: normal, statistic: median
Lower bound: -0.04016
Upper bound: 0.0894

Distribution: exponential, statistic: mean
Lower bound: 0.94664
Upper bound: 1.07713

Distribution: exponential, statistic: median
Lower bound: 0.6681
Upper bound: 0.77898

Distribution: mixed normal, statistic: mean
Lower bound: -0.11882
Upper bound: 0.15652

Distribution: mixed normal, statistic: median
Lower bound: -0.60593
Upper bound: 0.66746



# Part 3

Calculate power of t-test and Mann-Whitney test for different distributions and effect.  Separately cover case when means are equals and variances are very different

In [7]:
def calculate_power(gen_func, stat_crit, mu_init=0, sigma_init=1, mu_effect=0, sigma_effect=0, size=100, n_iterations=100):
    p_vals = []
    for i in range(n_iterations):
        if gen_func == 'normal':
            data1 = np.random.normal(loc=mu_init, scale=sigma_init, size=size)
            data2 = np.random.normal(loc=mu_init + mu_effect , scale=sigma_init + sigma_effect, size=size)
        else:
            data1 = np.random.exponential(scale=sigma_init, size=size)
            data2 = np.random.exponential(scale=sigma_init + sigma_effect, size=size)
            
        res = stat_crit(data1, data2, alternative='two-sided')
        p_vals.append(res.pvalue)
        
    p_vals = np.array(p_vals)
    power_test = p_vals[p_vals < ALPHA].shape[0] / p_vals.shape[0] * 100
    print(f"Criteria power is {power_test}")
    return p_vals

In [8]:
mu_effects = [0, 1]
sigma_effects = [0, 100]
criterions = [{'func': ttest_ind, 'name': 'T-test'},
              {'func': mannwhitneyu, 'name': 'Mann-Whitney'},
             ]

for mu in mu_effects:
    for sigma in sigma_effects:
        for criterion in criterions:
            print(f"{criterion['name']} Criteria for normal distribution with mu effect={mu}, sigma effect={sigma}")
            calculate_power('normal', criterion['func'] , mu_effect=mu, sigma_effect=sigma)
            print()
            
            print(f"{criterion['name']} Criteria for exponential distribution with sigma effect={sigma}")
            calculate_power('exponential', criterion['func'] , sigma_effect=sigma)
            print()

T-test Criteria for normal distribution with mu effect=0, sigma effect=0
Criteria power is 8.0

T-test Criteria for exponential distribution with sigma effect=0
Criteria power is 6.0

Mann-Whitney Criteria for normal distribution with mu effect=0, sigma effect=0
Criteria power is 5.0

Mann-Whitney Criteria for exponential distribution with sigma effect=0
Criteria power is 5.0

T-test Criteria for normal distribution with mu effect=0, sigma effect=100
Criteria power is 3.0

T-test Criteria for exponential distribution with sigma effect=100
Criteria power is 100.0

Mann-Whitney Criteria for normal distribution with mu effect=0, sigma effect=100
Criteria power is 12.0

Mann-Whitney Criteria for exponential distribution with sigma effect=100
Criteria power is 100.0

T-test Criteria for normal distribution with mu effect=1, sigma effect=0
Criteria power is 100.0

T-test Criteria for exponential distribution with sigma effect=0
Criteria power is 3.0

Mann-Whitney Criteria for normal distribu

# Part 4

In [9]:
MU = 0
SIGMA = 1
DELTA = 3
SAMPLE_SIZE = 1000
ALPHA = 0.05

In [10]:
def test_correct(distribution, sample_size=SAMPLE_SIZE, alpha=ALPHA):
    if distribution == 'normal':
        sample1 = np.random.normal(loc=MU, scale=SIGMA, size=sample_size)
        sample2 = np.random.normal(loc=MU, scale=SIGMA, size=sample_size)
    elif distribution == 'exponential':
        sample1 = np.random.exponential(scale=SIGMA, size=sample_size)
        sample2 = np.random.exponential(scale=SIGMA, size=sample_size)
    else:
        sample1 = np.concatenate([
            np.random.normal(loc=MU, scale=SIGMA, size=sample_size // 2),
            np.random.normal(loc=MU + DELTA, scale=SIGMA, size=sample_size // 2)
        ])
        sample2 = np.concatenate([
            np.random.normal(loc=MU, scale=SIGMA, size=sample_size // 2),
            np.random.normal(loc=MU + DELTA, scale=SIGMA, size=sample_size // 2)
        ])
        
    _, p_value_ttest = ttest_ind(sample1, sample2)
    _, p_value_mannwhitney = mannwhitneyu(sample1, sample2, alternative='two-sided')

    ttest_correct = (p_value_ttest >= alpha)
    mannwhitney_correct = (p_value_mannwhitney >= alpha)

    return ttest_correct, mannwhitney_correct

In [11]:
sample_size = 100
alpha = 0.05

distributions = ['normal', 'exponential', 'mixed normal']
n_tests = 1000

for distribution in distributions:
    ttest_correct_cnt = 0
    mannwhitney_correct_cnt = 0

    for _ in range(n_tests):
        ttest_correct, mannwhitney_correct = test_correct(distribution, sample_size, alpha)
        ttest_correct_cnt += ttest_correct
        mannwhitney_correct_cnt += mannwhitney_correct

    ttest_correct_res = (ttest_correct_cnt / n_tests) * 100
    mannwhitney_correct_res = (mannwhitney_correct_cnt / n_tests) * 100

    print(f"Distribution: {distribution}")
    print(f"T-Test Correctness: {round(ttest_correct_res, 2)}%")
    print(f"Mann-Whitney Correctness: {round(mannwhitney_correct_res, 2)}%")
    print()

Distribution: normal
T-Test Correctness: 94.6%
Mann-Whitney Correctness: 94.8%

Distribution: exponential
T-Test Correctness: 94.9%
Mann-Whitney Correctness: 94.7%

Distribution: mixed normal
T-Test Correctness: 99.9%
Mann-Whitney Correctness: 99.9%

